In [4]:
from sklearn.model_selection import ParameterGrid
from ultralytics import YOLO
import json
import os
import shutil
import random
import cv2
import matplotlib.pyplot as plt

In [5]:
json_path = "annotations.json"
images_dir = "images"
labels_dir = "labels"
data_yaml = "data.yaml"
train_dir = "dataset/train"
val_dir = "dataset/val"
os.makedirs(train_dir + "/images", exist_ok=True)
os.makedirs(train_dir + "/labels", exist_ok=True)
os.makedirs(val_dir + "/images", exist_ok=True)
os.makedirs(val_dir + "/labels", exist_ok=True)

In [6]:
# Load JSON annotation file
with open(json_path, "r") as f:
    data = json.load(f)

file_names = data["train"]["file_names"]
rois_list = data["train"]["rois_list"]
occupancy_list = data["train"]["occupancy_list"]

# Split dataset (80% train, 20% val)
data_pairs = list(zip(file_names, rois_list, occupancy_list))
random.shuffle(data_pairs)
split_idx = int(0.8 * len(data_pairs))
train_data = data_pairs[:split_idx]
val_data = data_pairs[split_idx:]

In [7]:
# Function to process and save labels
def save_labels(file_name, rois, occupancy, split):
    label_path = f"dataset/{split}/labels/" + file_name.replace(".JPG", ".txt")
    img_path = os.path.join(images_dir, file_name)
    shutil.copy(img_path, f"dataset/{split}/images/")
    
    with open(label_path, "w") as lf:
        for obj, occupied in zip(rois, occupancy):
            x_values = [p[0] for p in obj]
            y_values = [p[1] for p in obj]
            x_center = sum(x_values) / len(x_values)
            y_center = sum(y_values) / len(y_values)
            width = max(x_values) - min(x_values)
            height = max(y_values) - min(y_values)
            class_id = 0 if occupied else 1
            lf.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process all data
for file_name, rois, occupancy in train_data:
    save_labels(file_name, rois, occupancy, "train")
for file_name, rois, occupancy in val_data:
    save_labels(file_name, rois, occupancy, "val")

In [8]:
# Write data.yaml
with open(data_yaml, "w") as f:
    f.write("train: dataset/train\n")
    f.write("val: dataset/val\n")
    f.write("nc: 2\n")
    f.write("names: ['car', 'empty']\n")

In [9]:
weights = YOLO('./weights/weights11.pt')
print(weights)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [10]:
# Initialize the YOLO model
model = weights

# Best parameters found: {'lr0': 0.01, 'optimizer': 'Adam', 'weight_decay': 0.001}
results = model.train(
    data=data_yaml, 
    epochs=100,  # Larger number of epochs for final training
    imgsz=640,
    batch=16,
    optimizer='Adam',
    lr0=0.01,
    weight_decay=0.001
)

Ultralytics 8.3.96  Python-3.12.10 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i7-1260P)
engine\trainer: task=detect, mode=train, model=./weights/weights11.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train51, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

RuntimeError: Dataset 'data.yaml' error  
Dataset 'data.yaml' images not found , missing path 'C:\Users\abhir\ece-capstone\ML Model\.ipynb_checkpoints\datasets\dataset\val'
Note dataset download directory is 'C:\Users\abhir\ece-capstone\ML Model\.ipynb_checkpoints\datasets'. You can update this in 'C:\Users\abhir\AppData\Roaming\Ultralytics\settings.json'

In [8]:
# Visualize sample images
def visualize_samples(split, num=5):
    img_dir = f"dataset/{split}/images"
    label_dir = f"dataset/{split}/labels"

    sample_images = random.sample(os.listdir(img_dir), num)
    
    for img_file in sample_images:
        img_path = os.path.join(img_dir, img_file)
        label_path = os.path.join(label_dir, img_file.replace(".JPG", ".txt"))

        if not os.path.exists(img_path) or not os.path.exists(label_path):
            print(f"Skipping {img_file}, missing label or image file!")
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Could not load image {img_path}")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        h_img, w_img, _ = img.shape
        print(f"Processing {img_file} | Shape: {img.shape}")

        with open(label_path, "r") as lf:
            labels = lf.readlines()
            print(f"Label contents for {img_file}:\n{labels}")

            for label in labels:
                class_id, x, y, w, h = map(float, label.split())

                # Convert YOLO format (normalized) to pixel coordinates
                x1 = int((x - w / 2) * w_img)
                y1 = int((y - h / 2) * h_img)
                x2 = int((x + w / 2) * w_img)
                y2 = int((y + h / 2) * h_img)

                # Ensure box stays within image bounds
                x1 = max(0, min(w_img, x1))
                y1 = max(0, min(h_img, y1))
                x2 = max(0, min(w_img, x2))
                y2 = max(0, min(h_img, y2))

                # Ignore tiny bounding boxes
                min_box_size = 10  # Minimum pixels
                if (x2 - x1) < min_box_size or (y2 - y1) < min_box_size:
                    print(f"Skipping small box: {x1, y1, x2, y2}")
                    continue

                color = (255, 0, 0) if class_id == 0 else (0, 255, 0)
                label_name = "Car" if class_id == 0 else "Empty"

                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, label_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        plt.figure(figsize=(10, 10))  # Enlarge plot size
        plt.imshow(img)
        plt.axis("off")
        plt.show(block=True)  # Ensure the image is displayed

# Run and debug visualization
visualize_samples("train")

Processing GOPR6796.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6796.JPG:
['0 0.0754354375 0.806524 0.09533325 0.2688886666666668\n', '0 0.1642686875 0.757524 0.11099999999999999 0.27022233333333345\n', '1 0.2604354375 0.70274625 0.14300000000000002 0.2684443333333334\n', '0 0.36035206249999996 0.6427462500000001 0.16366675000000003 0.24622200000000005\n', '1 0.45443543750000004 0.5831905833333333 0.17666675000000004 0.21644433333333346\n', '0 0.5366854375 0.52930175 0.1749997499999999 0.18622233333333332\n', '1 0.6063520625000001 0.4829685 0.16166649999999994 0.15511133333333338\n', '0 0.66101875 0.44663508333333335 0.1513335 0.12488900000000008\n', '1 0.704352125 0.41674625 0.14100000000000001 0.11022233333333342\n', '0 0.05133125 0.49824633333333335 0.06225 0.10966666666666669\n', '0 0.10020625 0.46799633333333335 0.06425000000000002 0.10633333333333328\n', '0 0.15339375 0.43674633333333335 0.05725000000000001 0.10333333333333333\n', '0 0.20970625 0.4053296666666667 0.064 0.

<Figure size 1000x1000 with 1 Axes>

Processing GOPR6635.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6635.JPG:
['1 0.38941625 0.37298691666666667 0.15899999999999997 0.10400000000000004\n', '1 0.33291625 0.43376475000000003 0.1726665 0.13511099999999998\n', '1 0.263332875 0.5145424166666667 0.179 0.18222233333333338\n', '0 0.191207875 0.6070979999999999 0.16333325000000004 0.21244433333333324\n', '1 0.1177495625 0.7074313333333333 0.15733325000000004 0.2617776666666666\n', '0 0.435275 0.32416 0.15008350000000004 0.08733333333333332\n', '0 0.47342074999999995 0.28446541666666664 0.13549999999999995 0.06911133333333336\n', '1 0.534004 0.23118766666666668 0.11800000000000005 0.05222233333333334\n', '1 0.75679575 0.317771 0.11449999999999994 0.08200000000000002\n', '1 0.75385825 0.3531876666666667 0.127 0.09466666666666668\n', '1 0.7498790625 0.3978126666666667 0.14483325000000002 0.11683333333333334\n', '1 0.743254 0.46140983333333335 0.17000000000000004 0.15466666666666667\n', '1 0.7304206875 0.5421876666666667 0.19

<Figure size 1000x1000 with 1 Axes>

Processing GOPR6774.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6774.JPG:
['0 0.61550925 0.9193344999999999 0.304 0.22666433333333336\n', '1 0.7325092499999999 0.8218350833333333 0.259 0.33999766666666664\n', '0 0.78838425 0.705669 0.21899999999999997 0.2833333333333333\n', '1 0.82525925 0.6118356666666667 0.18400000000000005 0.236\n', '0 0.8499260000000001 0.5362245 0.16066675000000008 0.19200000000000006\n', '0 0.8669260625 0.4771133333333333 0.14100000000000001 0.14666666666666667\n', '0 0.878134375 0.43478 0.124 0.12422199999999994\n', '0 0.8864260625 0.4003910833333333 0.11183325 0.11022233333333326\n', '0 0.892676 0.37166883333333334 0.10083324999999999 0.09333300000000005\n', '0 0.8975926875000001 0.3491133333333333 0.09216674999999996 0.08311099999999999\n', '0 0.901301 0.3307800833333333 0.08466675000000001 0.0735553333333333\n', '0 0.904301 0.31494675 0.07850000000000001 0.06866666666666671\n', '1 0.0795251875 0.6999255833333333 0.1005 0.273111\n', '1 0.1557751875 0.6

<Figure size 1000x1000 with 1 Axes>

Processing GOPR6807.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6807.JPG:
['0 0.108282 0.5420463333333333 0.09966675 0.1933333333333333\n', '0 0.13794862500000002 0.4110463333333333 0.08400000000000003 0.12222199999999994\n', '1 0.1963653125 0.3848241666666667 0.0766665 0.1213333333333333\n', '1 0.179782 0.511713 0.09166650000000001 0.19199966666666674\n', '0 0.2582819375 0.4806018333333333 0.09199999999999997 0.18755566666666662\n', '0 0.259615375 0.35893525 0.07200000000000001 0.11244433333333331\n', '0 0.3426986875 0.4490463333333333 0.10966675000000004 0.1773333333333333\n', '0 0.427365375 0.4191575 0.13166649999999996 0.16400000000000003\n', '0 0.325782 0.33560191666666667 0.07566649999999997 0.100889\n', '1 0.391865375 0.3148241666666667 0.09166675000000002 0.09377766666666665\n', '0 0.454948625 0.296713 0.09966674999999997 0.08400000000000002\n', '1 0.507365375 0.39182408333333335 0.13966675 0.14755566666666664\n', '0 0.5786986875 0.37037966666666666 0.1433335 0.13022233

<Figure size 1000x1000 with 1 Axes>

Processing GOPR6662.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6662.JPG:
['1 0.9402329375 0.7279633333333333 0.12052275000000001 0.256\n', '0 0.76360225 0.8312966666666666 0.23299999999999998 0.29666666666666663\n', '0 0.70910225 0.6819633333333334 0.22999999999999998 0.2486666666666666\n', '0 0.6570189375 0.5469633333333334 0.22299999999999998 0.18800000000000006\n', '0 0.6117272500000001 0.43713 0.20800000000000007 0.13733333333333336\n', '0 0.5747273125 0.35190775 0.18966675 0.10400000000000004\n', '0 0.5446856875 0.2880745 0.17099999999999999 0.07644400000000001\n', '0 0.51918575 0.23985208333333333 0.15583324999999998 0.06088899999999997\n', '0 0.16396968750000002 0.8541955833333333 0.2113635 0.20707066666666663\n', '0 0.15995593749999998 0.6953025 0.19277124999999998 0.18232933333333323\n', '0 0.1641758125 0.567597 0.181518 0.17455400000000004\n', '0 0.17750931250000002 0.4524019166666667 0.1663535 0.12722866666666666\n', '1 0.21479624999999997 0.27213333333333334 0.1229

<Figure size 1000x1000 with 1 Axes>